In [1]:
import pandas as pd
import os
import datetime as dt
import re
import xlsxwriter
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px

In [2]:
def add_zero(x):
    """
    Функция добавляет нули для корректного отображения кода ресторана в четырехзначном формате: "0ХХХ"
    """
    x = str(x).strip()
    if len(x) == 1:
        x = '000' + x
    elif len(x) == 2:
        x = '00' + x
    elif len(x) == 3:
        x = '0' + x
    return x

In [3]:
daily = pd.read_excel('Обратная связь.xlsx', 
                      sheet_name="Основной лист", 
                      parse_dates=False, 
                      engine='openpyxl',
                      keep_default_na=True, 
                      convert_float=True, 
                      dtype={"Ресторан № (собеседование)": str, "Ресторан № (работа)": str}, 
                      na_values=[pd.NaT])

# Предобработка информации

# Убирает пустые строки в конце документа
daily = daily.dropna(how='all', thresh=6)
# Приводит номера ресторанов в корректный формат
daily['Ресторан № (собеседование)'] = daily['Ресторан № (собеседование)'].apply(add_zero)
daily['Ресторан № (работа)'] = daily['Ресторан № (работа)'].apply(add_zero)
# Удаляет лишние или заменяет символы для приведения информации колонок в единый формат
daily['Дивизион'] = daily['Дивизион'].apply(str.strip).apply(str.upper)
daily['Должность'] = daily['Должность'].str.replace('-', ' ')

In [4]:
# Группируем информацию по дате телефонного интерьвю и Дивизиону. Аггерерируем по кол-ву направленных
divs_groupped = daily.groupby(['Телефонное интервью (число)', 'Дивизион'], as_index=False) \
                        .aggregate({'ФИО': 'count'}) \
                        .rename(columns={'ФИО': 'направлено от ОП'}) 

In [5]:
# День отчета
day = '2021-01-14'

In [6]:
# Проверяем список должностей. 
# Важно, чтобы название каждой должности было написано без ошибок и данные распределились корректно
daily['Должность'].unique()

array(['Кассир/повар', 'Ночной уборщик', 'Водитель курьер', 'Тренер',
       'Дневной уборщик', nan, 'Кассир'], dtype=object)

In [7]:
# Группируем данные по Дивизиону и Должности
positions_groupped = daily.groupby(['Телефонное интервью (число)', 'Дивизион', 'Должность'], as_index=False) \
                        .aggregate({'ФИО': 'count'}) \
                        .rename(columns={'ФИО': 'направлено от ОП'}) 
# Выбираем данные за нужный день для создания отчета
day_positions_groupped = positions_groupped[positions_groupped['Телефонное интервью (число)'] == day]

In [14]:
# Распределение по дивизионам. Конкретные числа по направленным можно увидеть, наведя курсор на блок
fig = px.bar(day_positions_groupped, 
             x='Дивизион', 
             y='направлено от ОП', 
             color='Должность', 
             title=f"Распределение направленных по Дивизионам {day}")
fig.show()

In [9]:
# Создание pie-charts по дивизионам в цикле

div_lst = day_positions_groupped['Дивизион'].unique()

for div in div_lst:
    div_daily_info = day_positions_groupped[day_positions_groupped['Дивизион'] == div]
    all_sent = div_daily_info['направлено от ОП'].sum()
    fig = px.pie(div_daily_info, 
                 values='направлено от ОП', 
                 names='Должность', 
                 title=f'{day} {div}: pаспределение направленных по должностям. Всего направленных: {all_sent}')
    fig.update_traces(textinfo='value', textfont_size=20)
    fig.show()